# Credito Bancario

# Contenido <a id='back'></a>

* [Análisis del riesgo de incumplimiento de los prestatarios](#intro)
* [Etapa 1. Abre el archivo de datos y mira la información general](#data_review)
* [Etapa 2. Exploración de datos](#exploration)
* [Etapa 3. Transformación de datos](#transformation)
    * [3.1 Restaurar valores en total_income](#total_income)
    * [3.2 Restaurar valores en days_employed](#days_employed)
* [Etapa 4. Clasificación de datos](#classification)    
* [Etapa 5. Comprobación de las hipótesis](#hypotheses)  
* [Conclusión general](#end)

# Análisis del riesgo de incumplimiento de los prestatarios

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Se Deberá averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Abrir el archivo de datos y mirar la información general. 

In [1]:
import pandas as pd

In [3]:
banco = pd.read_csv("datasets/credit_scoring_eng.csv")

## Exploración de datos <a id="exploration"></a>

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Observaremos cuántas filas y columnas tiene nuestro conjunto de datos

In [4]:
banco.shape

(21525, 12)

In [5]:
banco.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Se observan datos faltantes tanto en la columna "days_empoyed" como en la columna "total_income". Además se observan valores negativos y decimales irrelevantes en "days_employed" junto a diferencia de nomenclatura en la columna "education". Hay que proceder revisando la existencia de más datos faltantes en el resto de columnas, unificar nomenclatura y averiguar a que podría corresponder la existencia de valores negativos en "days_employed".

In [6]:
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Se observan valores ausentes solamente en las columnas "days_employed" y "total_income", tal como se observó en las primeras 15 filas del DataSet anteriormente.

In [7]:
banco_filtered_de = banco[banco["days_employed"].isna()==True]
banco_filtered_de.head(60)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


A primera vista, observando las primeras 60 filas del Dataframe filtrado con los valores ausentes de la columna "days_employed" se podría inferir que estos son simétricos con los datos ausentes de la columna "total_income", pero al ser una muestra pequeña en comparación a la cantidad de filas con datos ausentes, alrededor de un 3% de las filas con datos ausentes, no se puede asegurar esta observación. Lo más apropiado seria continuar comprobando si la ausencia de datos en ambas columnas es simétrica o no.

In [8]:
print(f'{"Tamaño Dataframe banco filtrado por ausentes en days_employed:"} {banco_filtered_de.shape}')
banco_refiltered_ti = banco_filtered_de[banco_filtered_de["total_income"].isna()==True]
print(f'{"Tamaño Dataframe banco filtrado vuelto a filtrar por ausentes en total_income:"} {banco_refiltered_ti.shape}')

Tamaño Dataframe banco filtrado por ausentes en days_employed: (2174, 12)
Tamaño Dataframe banco filtrado vuelto a filtrar por ausentes en total_income: (2174, 12)


**Conclusión intermedia**

El número de filas de la tabla filtrada por los valores ausentes en days_employed coincide con los de la tabla vuelta a filtrar con los valores ausentes en total_income, con esto podemos concluir que los valores ausentes en ambas columnas son simétricos y se presentan en las mismas filas del Dataframe.

Lo valores ausentes se encuentran en 2174 filas, lo que corresponde a un 10% de las filas del conjunto de datos por lo que debido a la gran cantidad de datos ausentes se procederá a buscar si existe alguna tendencia con respecto a otras columnas o si la ausencia de valores es aleatoria. En primer lugar se abordara que la característica que define esta ausencia de datos es el nivel de estudios de las personas, debido a que un menor nivel de estudios podría incidir en una menor experiencia laboral y un menor ingreso por lo que gente con menor nivel de estudios decidió omitir estos datos. Se procederá a comparar la distribución de datos en el Dataframe completo versus el Dataframe compuesto únicamente de las filas con datos ausentes con respecto al nivel de estudios de las personas y se observara si existe alguna diferencia en la distribución, de existir diferencias se concluiría que esa característica influye en la ausencia de datos.

In [9]:
banco_filtered_de["education"].head(15)

12     secondary education
26     secondary education
29     secondary education
41     secondary education
55     secondary education
65     secondary education
67       bachelor's degree
72       bachelor's degree
82       bachelor's degree
83     secondary education
90       bachelor's degree
94       bachelor's degree
96     SECONDARY EDUCATION
97       bachelor's degree
120    secondary education
Name: education, dtype: object

In [10]:
banco_filtered_de["education"].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

Independiente de los problemas de nomenclaturas existentes, se puede observar por un lado que la mayor parte de los datos ausentes se encuentran en personas que tienen como máximo estudios secundarios lo que podría ir de la mano con la hipótesis antes planteada, pero por otra parte muy poca gente con menos estudios que los anteriormente descritos tiene datos ausentes. 

De momento no se podría atribuir la ausencia de datos a la característica del nivel de educación de las personas, para que la hipótesis fuera correcta la cantidad de personas con datos ausentes debería disminuir según aumenta el nivel de educación y aunque es así para la mayoría de los niveles educativos mostrados no lo es para la eduación primaria la cual es el nivel educativo más bajo de todos. Se procederá a comprobar la distribución del Dataframe entero para ver si existe algún cambio con respecto al recién analizado o si por el contrario es similar y por ende este no está afectando a la ausencia de datos

In [11]:
banco["education"].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

**Conclusión intermedia**

La distribución en el Dataframe filtrado y entero con respecto al nivel educativo es coincidente por lo cual podemos descartar que esta característica influya de alguna forma en la ausencia de datos para estas personas.

Se procederá a comprobar si alguna otra característica puede influir en la ausencia de datos, específicamente "income_type" ya que el tipo de empleo puede influir en la ausencia de datos con respecto al ingreso mensual, en caso de que este último no muestre indicios de afectar la ausencia de datos se realizará una comprobación del resto de columnas exceptuando las que poseen datos faltantes y dejando de lado también "dob_years" y "purpose" debido a la gran diversificación de características que estas poseen.

In [12]:
banco_filtered_de["income_type"].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [13]:
banco["income_type"].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

**Conclusión intermedia**

Al comprobar la distribución de "income_type" se puede observar que tampoco existe alguna diferencia entre la distribución del Dataframe entero y el filtrado, por lo que aumenta la posibilidad de que la ausencia de datos sea aleatoria. Se procederá a realizar una comprobación rápida del resto de las columnas dejando de lado las anteriormente mencionadas para llegar a una conclusión final con respecto a la aleatoriedad de los datos ausentes**

In [20]:
dataframes = [banco_filtered_de, banco]
columnas = ["debt", "gender", "family_status", "children"]
banco.name = 'completo'
banco_filtered_de.name = 'filtrado'
for column in columnas:
    for df in dataframes:
        print(df.name)
        print(df[column].value_counts(normalize=True))
    print("")

filtrado
0    0.921803
1    0.078197
Name: debt, dtype: float64
completo
0    0.919117
1    0.080883
Name: debt, dtype: float64

filtrado
F    0.682613
M    0.317387
Name: gender, dtype: float64
completo
F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64

filtrado
married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64
completo
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64

filtrado
 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64
completo
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype:

**Conclusiones**

Después de revisar las distribuciones con respecto a diversas características y no encontrar ningún patrón entre ellas, podemos concluir que la ausencia de datos en el Dataframe es completamente aleatoria sin verse afectada por alguna columna en particular.

Debido a la gran cantidad de valores ausentes y por ende la incapacidad de eliminar estas filas por lo determinantes que serían en el análisis final, se procederá a rellenar estos valores con la media o mediana, según se determine más adelante, de la característica que sea más representativa de estos, para lo cual se realizará otro análisis.

Se procederá a analizar cada columna, una por una, buscando diversos problemas como los identificados al inicio del informe, es decir valores sin sentido, problemas de nomenclaturas en los registros, entre otros. Posteriormente se procederá a corregir cada problema según sea el caso.

## Transformación de datos <a id="transformation"></a>

Observaremos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente

In [23]:
banco["education"].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [24]:
banco["education"] = banco["education"].str.lower()

Comprobaremos todos los valores en la columna para asegurarnos de que los hayamos corregido

In [26]:
banco["education"].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Comprobaremos los datos en la columna 'children'

In [27]:
banco["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Existen valores negativos en la cantidad de hijos que posee cada persona lo cual es imposible. El porcentaje de datos incorrectos es de un 0,2% por lo que lo más probable es que estos se hayan producido debido a un error humano durante el ingreso de los datos. Como la cantidad de datos erróneos es muy pequeña se procederá a reemplazarlos por su valor positivo sin que esto afecte la distribución de datos anteriormente mostrada. Por otra parte se observa el valor de 20 hijos lo cual es un valor altamente atípico, esto podria explicarse por un error de tipeo al ingresar los datos siendo el verdadero valor de estos de 2, se procederá a reemplazar los valores de 20 por 2.

In [28]:
banco["children"] = banco["children"].abs()
banco["children"] = banco["children"].replace(20,2)

Comprobaremos todos los valores en la columna para asegurarnos de que los hayamos corregido

In [29]:
banco["children"].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Como se mencionó al comienzo del informe la columna de "days_employed" posee valores negativos lo cual no es lógico con respecto a que la columna representa los días de experiencia que posee la persona, además que los valores cuentan con muchos decimales lo cual tampoco es relevante al ser la unidad de medida en días. Se comenzará comprobando que porcentaje de los datos posee el error de ser negativos.

In [30]:
days_negative = banco[banco["days_employed"] < 0]
percentaje_days = len(days_negative)/len(banco)
percentaje_format_days = "{:.2%}".format(percentaje_days)
print(percentaje_format_days)

73.90%


La cantidad de datos con problemas es muy alta por lo que supondremos que existió algún problema técnico durante el ingreso de estos datos y que su valor correcto es de signo positivo. Ya que no podemos eliminar estas filas puesto que componen prácticamente tres cuartos de los datos, se procederá a reemplazar estos valores por valores de la misma magnitud, pero positivos y de la misma forma se convertirán todos los valores a números enteros.

In [31]:
banco["days_employed"] = banco["days_employed"].abs()
banco["days_employed"] = banco["days_employed"].round()

Comprobaremos los resultados

In [33]:
banco.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8438.0,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4025.0,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.0,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4125.0,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,2879.0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,153.0,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2189.0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Ahora observaremos la edad de los clientes

In [34]:
banco["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [35]:
years_zero = banco[banco["dob_years"] == 0]
percentaje_years = len(years_zero)/len(banco)
percentaje_format = "{:.2%}".format(percentaje_years)
print(percentaje_format)

0.47%


Se puede observar un valor de 0 lo cual no tiene sentido para la edad de las personas del Dataframe. Debido a que el porcentaje de filas con este error es muy baja, se procederá a reemplazar los valores de 0 por la mediana de los años, ya que la media de los datos puede arrojar valores no enteros lo cual no es apto para este tipo de dato, excluyendo a los años con el valor erróneo en sus datos.

In [36]:
banco_not_zero = banco[banco["dob_years"]>0]
median_years = banco_not_zero["dob_years"].median()
banco["dob_years"] = banco["dob_years"].replace(0,int(median_years))

In [37]:
banco["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

Ahora revisemos la columna `family_status`.

In [38]:
banco["family_status"].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

No se aprecian valores poblemáticos en la columna de "family_status" por lo que no se procederá a intervenir.

Ahora revisaremos la columna `gender`.

In [39]:
banco["gender"].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [40]:
banco["gender"].value_counts()


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Se procederá a uniformar la nomenclatura con respecto al resto del Dataframe transformando los caracteres a minúsculas, también se reemplazará "XNA" por "unknown" ya que el primer término no nos dice mucho.

In [41]:
banco["gender"] = banco["gender"].str.lower()
banco["gender"] = banco["gender"].replace("xna","unknown")

In [42]:
banco["gender"].unique()

array(['f', 'm', 'unknown'], dtype=object)

Ahora vamos a revisar la columna `income_type`.

In [43]:
banco["income_type"].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

No se aprecian valores poblemáticos en la columna de "income_type" por lo que no se procederá a intervenir

De existir datos duplicados, en una primera instancia al no contar con un identificador único para cada fila tal como un id propio o pasaporte de la persona podría pensarse que las filas duplicadas pueden ser mera coincidencia, pero en el caso actual en el que existen múltiples columnas que reunen distintos tipos de datos personales, la probabilidad que dos personas distintas coincidan en todas estas es muy baja, por lo que si el porcentaje de datos repetido es muy bajo se procederá a eliminarlos.

In [44]:
duplicated_banco = banco[banco.duplicated()]
percentaje_duplicated = len(duplicated_banco)/len(banco)
percentaje_format = "{:.2%}".format(percentaje_duplicated)
print(percentaje_format)

0.33%


Se abordarán los datos duplicados eliminandolos.

In [46]:
banco = banco.drop_duplicates().reset_index(drop = True)

Comprobaremos el resultado.

In [47]:
duplicated_banco = banco[banco.duplicated()]
len(duplicated_banco)

0

In [48]:
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Se procedió a eliminar los datos duplicados del conjunto de datos, esto conllevó a una disminución de las filas totales del conjunto de 21525 a 21454, es decir una disminución de un 0,3% de las filas aprox. Por otra parte, se puede observar que la cantidad de datos en las columnas que presentan ausencias se mantiene en 19351 por lo que se concluye que ninguna de las filas duplicadas eliminadas correspondian a una fila con ausencia de datos.

# Trabajar con valores ausentes 

Observaremos los diccionarios proporcionados en la base de datos.

In [49]:
banco[["education","education_id"]]

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,secondary education,1
3,secondary education,1
4,secondary education,1
...,...,...
21449,secondary education,1
21450,secondary education,1
21451,secondary education,1
21452,secondary education,1


In [50]:
banco[["family_status","family_status_id"]]

,family_status,family_status_id
0,married,0
1,married,0
2,married,0
3,married,0
4,civil partnership,1
...,...,...
21449,civil partnership,1
21450,married,0
21451,civil partnership,1
21452,married,0


Se revisaron los diccionarios en que se proporcionan IDs para las columnas "education" y "family_status, se decidió para este proyecto trabajar con los diccionarios con los nombres completo para poder realizar un análisis más detallado en función de estas características.

### Restaurar valores ausentes en `total_income` <a id="total_income"></a>

Se procederá a crear una categoría de grupos de edades, como no tenemos mayor información sobre requerimientos de esta categoría se crearán grupos de 10 años, partiendo del mínimo posible por si se requiriera en algún momento el ingreso de una edad menor.

In [51]:
# Vamos a escribir una función que calcule la categoría de edad
def age_group(age):
    if 1 <= age < 10:
        return "1-9"
    elif 10 <= age < 20:
        return "10-19"
    elif 20 <= age < 30:
        return "20-29"
    elif 30 <= age < 40:
        return "30-39"
    elif 40 <= age < 50:
        return "40-49"
    elif 50 <= age < 60:
        return "50-59"
    return "60+" 

Se probará la función anteriormente creada.

In [52]:
age_group(40)

'40-49'

Se creará una nueva columna basada en la función.

In [54]:
banco["age_group"] = banco["dob_years"].apply(age_group)

Observaremos la nueva columna en el dataframe.

In [55]:
banco.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.102,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.802,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.752,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.568,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.572,to have a wedding,50-59


Los factores que podrían determinar los ingresos de las personas son principalmente el tipo de empleo, "income_type", ya que de esto depende principalmente cuanto le pagarán a la persona seguido de la educación, "education", ya que puede existir una correlación entre un mayor nivel de estudios con un mayor salario para la persona y en tercer lugar podría existir alguna correlación relacionada a la edad de las personas, "dob_years". Se procederá a observar cual característica es la más representativa y sobre esta se buscará la existencia de valores extremos para decidir si utilizar su media o mediana.

Crearemos una tabla sin valores ausentes.

In [56]:
banco_no_missing = banco.dropna()
banco_no_missing.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.102,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.802,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.752,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.568,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.572,to have a wedding,50-59
5,0,926.0,27,bachelor's degree,0,civil partnership,1,m,business,0,40922.170,purchase of the house,20-29
6,0,2879.0,43,bachelor's degree,0,married,0,f,business,0,38484.156,housing transactions,40-49
7,0,153.0,50,secondary education,1,married,0,m,employee,0,21731.829,education,50-59
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,f,employee,0,15337.093,having a wedding,30-39
9,0,2189.0,41,secondary education,1,married,0,m,employee,0,23108.150,purchase of the house for my family,40-49


Observaremos los valores medios de los ingresos en función de los factores identificados.

In [62]:
factores = ["income_type", "education", "age_group"]
for column in factores:
    for medida in ['mean', 'median']:
        print(medida)
        print(banco_no_missing.pivot_table(index=column, values="total_income", aggfunc=medida))
        print("")
    print("")

mean
                             total_income
income_type                              
business                     32386.793835
civil servant                27343.729582
employee                     25820.841683
entrepreneur                 79866.103000
paternity / maternity leave   8612.661000
retiree                      21940.394503
student                      15712.260000
unemployed                   21014.360500

median
                             total_income
income_type                              
business                       27577.2720
civil servant                  24071.6695
employee                       22815.1035
entrepreneur                   79866.1030
paternity / maternity leave     8612.6610
retiree                        18962.3180
student                        15712.2600
unemployed                     21014.3605


mean
                     total_income
education                        
bachelor's degree    33142.802434
graduate degree      27960.024667
prim

Después de observar las características que podrían definir de mejor manera los ingresos, se optó por elegir el tipo de empleo ya que es la principal característica directamente relacionada con el monto de ingreso. Con respecto a si usar la media o la mediana, se escogió esta última debido a que algunos tipos de empleo se ven afectados por valores atípicos que incrementan el valor de la media.

Escribiremos una función que usaremos para completar los valores ausentes.

In [63]:
def llenar(carac, mediana):
    banco.loc[banco["income_type"]==carac, "total_income"] = banco.loc[banco["income_type"]==carac]["total_income"].fillna(mediana)                

In [64]:
llenar("retiree", 18962.3180)
banco.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.102,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.802,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.752,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.568,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.572,to have a wedding,50-59
5,0,926.0,27,bachelor's degree,0,civil partnership,1,m,business,0,40922.170,purchase of the house,20-29
6,0,2879.0,43,bachelor's degree,0,married,0,f,business,0,38484.156,housing transactions,40-49
7,0,153.0,50,secondary education,1,married,0,m,employee,0,21731.829,education,50-59
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,f,employee,0,15337.093,having a wedding,30-39
9,0,2189.0,41,secondary education,1,married,0,m,employee,0,23108.150,purchase of the house for my family,40-49


Observando las primeras 30 filas podemos notar que el valor de "total income" de la fila 12 que corresponde a un "income_type" de "retiree" fue llenado con el valor de mediana elegido, mientras que el valor de "total_income" de la fila 26 correspondiente a un "income_type" de "civil_servant" continua ausente. Con lo anterior comprobamos que la función creada actúa tal como se espera, rellenando los valores ausentes con el valor escogido y solo de las filas que correspondan al "income_type" elegido.

In [66]:
income_pivot_median = banco_no_missing.pivot_table(index='income_type', values="total_income", aggfunc='median')
df_income_pivot_median = income_pivot_median.reset_index()
df_income_pivot_median.apply(lambda x: llenar(x["income_type"], x["total_income"]), axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

In [69]:
banco.head(60)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.1020,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.8020,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.7520,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.5680,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.5720,to have a wedding,50-59
5,0,926.0,27,bachelor's degree,0,civil partnership,1,m,business,0,40922.1700,purchase of the house,20-29
6,0,2879.0,43,bachelor's degree,0,married,0,f,business,0,38484.1560,housing transactions,40-49
7,0,153.0,50,secondary education,1,married,0,m,employee,0,21731.8290,education,50-59
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,f,employee,0,15337.0930,having a wedding,30-39
9,0,2189.0,41,secondary education,1,married,0,m,employee,0,23108.1500,purchase of the house for my family,40-49


In [68]:
banco["total_income"].isna().sum()

0

Se puede observar en las primeras 60 líneas que los valores ausentes de la columna "total_income" han sido completados con los valores elegidos dependiendo de la columna "income_type" correspondiente, también se comprobó el número de valores ausentes en la columna y estos son iguales a cero por lo que se han rellenado todos los valores ausentes.

In [70]:
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restaurar valores en `days_employed` <a id="days_employed"></a>

Con respecto a restaurar los valores ausentes en "days_employed" el único parámetro que se puede ver relacionado a esta característica es "dob_years" ya que a mayor edad de la persona más días de experiencia puede tener en su trabajo. Procederemos a revisar la distribución de datos para la media y mediana para poder decidir cual utilizar.

In [71]:
for medida in ['mean', 'median']:
    print(medida)
    print(banco_no_missing.pivot_table(index="age_group", values="days_employed", aggfunc=medida))
    print("")

mean
           days_employed
age_group               
10-19         633.615385
20-29        2089.058865
30-39        4155.028186
40-49       13439.223756
50-59      132907.547870
60+        286544.135255

median
           days_employed
age_group               
10-19              724.0
20-29             1006.0
30-39             1602.0
40-49             2109.0
50-59             4797.0
60+             355230.0



Se puede observar una gran disparidad entre los valores de la media y mediana de este parámetro por lo cual deducimos que existen valores extremadamente anómalos entre las filas, debido a esto se optará por utilizar la mediana para rellenar los valores ausentes.

Crearemos una función que calcule la mediana.

In [72]:
def llenar_edad(carac, mediana):
    banco.loc[banco["age_group"]==carac, "days_employed"] = banco.loc[banco["age_group"]==carac]["days_employed"].fillna(mediana) 

In [73]:
llenar_edad("60+", 355230)
banco.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.1020,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.8020,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.7520,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.5680,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.5720,to have a wedding,50-59
5,0,926.0,27,bachelor's degree,0,civil partnership,1,m,business,0,40922.1700,purchase of the house,20-29
6,0,2879.0,43,bachelor's degree,0,married,0,f,business,0,38484.1560,housing transactions,40-49
7,0,153.0,50,secondary education,1,married,0,m,employee,0,21731.8290,education,50-59
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,f,employee,0,15337.0930,having a wedding,30-39
9,0,2189.0,41,secondary education,1,married,0,m,employee,0,23108.1500,purchase of the house for my family,40-49


Observando las primeras 30 filas podemos notar que el valor de "days_employed" de la fila 12 que corresponde a un "age_group" de "60+" fue llenado con el valor de mediana elegido, mientras que el valor de "days_employed" de la fila 26 correspondiente a un "age_group" de "40-49" continua ausente. Con lo anterior comprobamos que la función creada actúa tal como se espera.

In [75]:
years_pivot_days_median = banco_no_missing.pivot_table(index='age_group', values="days_employed", aggfunc='median')
df_days_pivot_median = years_pivot_days_median.reset_index()
df_days_pivot_median.apply(lambda x: llenar_edad(x["age_group"], x["days_employed"]), axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
dtype: object

In [76]:
# Comprueba si la función funcionó
banco.head(60)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8438.0,42,bachelor's degree,0,married,0,f,employee,0,40620.1020,purchase of the house,40-49
1,1,4025.0,36,secondary education,1,married,0,f,employee,0,17932.8020,car purchase,30-39
2,0,5623.0,33,secondary education,1,married,0,m,employee,0,23341.7520,purchase of the house,30-39
3,3,4125.0,32,secondary education,1,married,0,m,employee,0,42820.5680,supplementary education,30-39
4,0,340266.0,53,secondary education,1,civil partnership,1,f,retiree,0,25378.5720,to have a wedding,50-59
5,0,926.0,27,bachelor's degree,0,civil partnership,1,m,business,0,40922.1700,purchase of the house,20-29
6,0,2879.0,43,bachelor's degree,0,married,0,f,business,0,38484.1560,housing transactions,40-49
7,0,153.0,50,secondary education,1,married,0,m,employee,0,21731.8290,education,50-59
8,2,6930.0,35,bachelor's degree,0,civil partnership,1,f,employee,0,15337.0930,having a wedding,30-39
9,0,2189.0,41,secondary education,1,married,0,m,employee,0,23108.1500,purchase of the house for my family,40-49


In [77]:
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Podemos observar que todos los valores ausentes se encuentran corregidos, por lo que podemos continuar con nuestro análisis.

## Clasificación de datos <a id="classification"></a>

Se procede a mostrar los valores de los datos seleccionados para la clasificación.

In [78]:
clasificacion_banco = banco[["debt","children","family_status","total_income","purpose"]]
clasificacion_banco

,debt,children,family_status,total_income,purpose
0,0,1,married,40620.102,purchase of the house
1,0,1,married,17932.802,car purchase
2,0,0,married,23341.752,purchase of the house
3,0,3,married,42820.568,supplementary education
4,0,0,civil partnership,25378.572,to have a wedding
...,...,...,...,...,...
21449,0,1,civil partnership,35966.698,housing transactions
21450,0,0,married,24959.969,purchase of a car
21451,1,1,civil partnership,14347.610,property
21452,1,3,married,39054.888,buying my own car


Comprobaremos los valores únicos.

In [79]:
banco["purpose"].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [80]:
banco["family_status"].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

En el caso de "purpose" se pueden observar muchos valores únicos lo cual dificulta el análisis de estos datos pero a su vez estos presentan temáticas comunes, por lo anterior se decide clasificarlos en 5 categorías temáticas, estas son "wedding", "housing", "commercial", "car" y "education". Por parte de "family_status" los valores únicos son lo suficientemente específicos y reducidos por lo que no se realizará una mayor clasificación de estos datos.


In [81]:
def purpose_group(row):    
    if "wedding" in row:
        return("wedding")
    elif "car" in row:
        return("car")
    elif "educat" in row or "university" in row:
        return("education")
    elif "transaction" in row or "commercial" in row or "renting" in row:
        return("commercial")
    else:
        return("housing") 

In [82]:
banco["purpose_group"] = banco["purpose"].apply(purpose_group)
banco["purpose_group"].value_counts()

housing       6895
car           4306
education     4013
commercial    3916
wedding       2324
Name: purpose_group, dtype: int64

Revisaremos todos los datos numéricos en la columna seleccionada para la clasificación.

In [83]:
banco["children"].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [84]:
banco["total_income"].value_counts()

22815.1035    1070
27577.2720     502
18962.3180     387
24071.6695     145
79866.1030       2
              ... 
27020.8950       1
23686.8350       1
9606.2940        1
28156.7620       1
13127.5870       1
Name: total_income, Length: 19350, dtype: int64

In [85]:
banco["total_income"].nunique()

19350

In [86]:
banco["total_income"].quantile(0.5)

22815.103499999997

In [87]:
banco["total_income"].quantile(0.75)

31331.347999999998

In [88]:
banco["total_income"].quantile(0.94)

50368.575639999995

Con respecto a la columna "children" observando la cantidad de valores respecto a la cantidad de hijos se decide agrupar desde los 3 hijos o más en una sola categoría debido a que son muchos menos datos en comparación a cantidades menores de hijos. Investigando la columna "total_income" es inviable utilizar los valores únicos por si mismos ya que estos ascienden a 19350 valores, por otra parte, podemos apreciar que la mayor acumulación de datos se da en un ingreso de hasta 50000, correspondiendo entre 0 y 50000 a aproximadamente un 94% de los datos, debido a esto y a que prácticamente el 99% de los datos solo se repite 1 o 2 veces es que se dividirán en tramos de 10000 y un último tramo de 50000 o más

Crearemos las funciones para realizar las agrupaciones.

In [89]:
def children_group(children):
    if children == 0:
        return "0"
    elif children == 1:
        return "1"
    elif children == 2:
        return "2"
    else:
        return "3+"

In [90]:
def income_group(income):
    if 0 <= income < 10000:
        return "0-9999"
    elif 10000 <= income < 20000:
        return "10000-19999"
    elif 20000 <= income < 30000:
        return "20000-29999"
    elif 30000 <= income < 40000:
        return "30000-39999"
    elif 40000 <= income < 50000:
        return "40000-49999"
    return "50000+" 

Aplicaremos las funciones y observaremos los resultados.

In [92]:
banco["children_group"] = banco["children"].apply(children_group)

In [93]:
banco["income_group"] = banco["total_income"].apply(income_group)

In [94]:
banco["children_group"].value_counts()

0     14091
1      4855
2      2128
3+      380
Name: children_group, dtype: int64

In [95]:
banco["income_group"].value_counts()

20000-29999    7779
10000-19999    6829
30000-39999    3107
40000-49999    1492
50000+         1321
0-9999          926
Name: income_group, dtype: int64

## Comprobación de las hipótesis <a id="hypotheses"></a>


¿Existe una correlación entre tener hijos y pagar a tiempo?

In [96]:
children_df = banco[["children_group","debt","children"]]
children_pivot = children_df.pivot_table(index="children_group",  columns="debt", aggfunc="count")
children_pivot

children      
debt                  0     1
children_group               
0                 13028  1063
1                  4410   445
2                  1926   202
3+                  349    31

In [97]:
children_percentage = children_pivot.div( children_pivot.iloc[:,0], axis=0 )
children_percentage

children          
debt                  0         1
children_group                   
0                   1.0  0.081593
1                   1.0  0.100907
2                   1.0  0.104881
3+                  1.0  0.088825

Considerando que la tasa de incumplimiento general de los datos es de alrededor de un 8,8%, se puede apreciar que la gente sin hijos tiende a tener un mejor ratio en el cumplimiento de pago a tiempo, también es de considerar que aunque todas las personas con hijos se encuentras sobre el promedio de incumplimiento general, las personas con 3 o más hijos presentan un ratio mejor de cumplimiento que las personas con 1 o 2 hijos. Lo primero puede deberse a que personas con hijos sean más proclives a adquirir deudas al poseer más gastos mientras que lo segundo podría haber sido influenciado por la poca cantidad de datos versus el resto de los segmentos. Finalmente se puede concluir que existe una correlación entre tener hijos y pagar a tiempo, presentándose una mayor tendencia en el pago a tiempo en gente sin hijos pero no puede confirmarse una correlación directa con el número de hijos que posea la persona, necesitando una mayor cantidad de datos para afirmar que a mayor cantidad de hijos disminuyera la tendencia a pagar a tiempo.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [98]:
family_df = banco[["family_status","debt","family_status_id"]]
family_pivot = family_df.pivot_table(index="family_status",  columns="debt", aggfunc="count")
family_pivot

family_status_id     
debt                             0    1
family_status                          
civil partnership             3763  388
divorced                      1110   85
married                      11408  931
unmarried                     2536  274
widow / widower                896   63

In [99]:
family_percentage = family_pivot.div( family_pivot.iloc[:,0], axis=0 )
family_percentage

family_status_id          
debt                             0         1
family_status                               
civil partnership              1.0  0.103109
divorced                       1.0  0.076577
married                        1.0  0.081609
unmarried                      1.0  0.108044
widow / widower                1.0  0.070312

El estado familiar fue una de las categorías que prácticamente no presento manipulaciones durante el desarrollo del informe, considerando lo anterior se puede observar que solo dos categorías presentan una tasa de incumplimiento por sobre la general, personas que no están casadas y personas con unión civil, de lo anterior podría inferirse que existe una mayor relación hacia la edad de las personas que a la situación familiar en si misma ya que la unión civil o le gente que aún no se casa se relaciona a gente más joven que el resto de las categorías.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [100]:
income_df = banco[["income_group","debt","total_income"]]
income_pivot = income_df.pivot_table(index="income_group",  columns="debt", aggfunc="count")
income_pivot

total_income     
debt                    0    1
income_group                  
0-9999                868   58
10000-19999          6244  585
20000-29999          7117  662
30000-39999          2865  242
40000-49999          1390  102
50000+               1229   92

In [101]:
income_percentage = income_pivot.div( income_pivot.iloc[:,0], axis=0 )
income_percentage

total_income          
debt                    0         1
income_group                       
0-9999                1.0  0.066820
10000-19999           1.0  0.093690
20000-29999           1.0  0.093017
30000-39999           1.0  0.084468
40000-49999           1.0  0.073381
50000+                1.0  0.074858

Con respecto al ingreso total de las personas se puede observar que existe una mayor tendencia al incumplimiento en los ingresos medios de la gente mientras que para las personas de ingreso más bajo o más alto disminuye la tasa de incumplimiento. En este caso se puede inferir que la gente de ingreso medio tenga un mayor acceso a créditos, ya que la gente de menor ingreso puede verse rechazada en el proceso de petición de los créditos mientras que gente de mayor ingreso no los necesite. En razón de lo anterior existe una mayor probabilidad de que la gente de ingresos medios no cumplan con el pago ya que son los que más acceso tienen a los préstamos y también concentran la mayor parte de la muestra en comparación con las otras categorías.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [102]:
purpose_df = banco[["purpose_group","debt","purpose"]]
purpose_pivot = purpose_df.pivot_table(index="purpose_group",  columns="debt", aggfunc="count")
purpose_pivot

purpose     
debt                0    1
purpose_group             
car              3903  403
commercial       3612  304
education        3643  370
housing          6417  478
wedding          2138  186

In [103]:
purpose_percentage = purpose_pivot.div( purpose_pivot.iloc[:,0], axis=0 )
purpose_percentage

purpose          
debt                0         1
purpose_group                  
car               1.0  0.103254
commercial        1.0  0.084164
education         1.0  0.101565
housing           1.0  0.074490
wedding           1.0  0.086997

Analizando el propósito del crédito con respecto a la tasa de incumplimiento se concluye que bajo la categorización realizada dos categorías quedan con una tasa de incumplimiento por sobre la tasa general, estos serían las relacionados con educación y la adquisición de un vehículo. El primero puede deberse a gente joven que busca adquirir un crédito para una primera educación superior o mejorar la que ya posee y por ende deben endeudarse con el fin de a futuro conseguir un trabajo estable que les permita pagar estos préstamos, con respecto a la segunda podría igualmente verse respaldada por gente joven que adquiere su primer vehículo y por tanto no tiene los medios para cumplir con el préstamo regularmente, aunque ambas conclusiones se verían más ligadas a la edad de la persona que al propósito mismo del préstamo

# Conclusión general <a id='end'></a> 

 Comenzando con los valores ausentes estos se encontraron en solo dos columnas del Dataframe, "days_employed" y "total_income", despues de un análisis de estos se pudo concluir que eran simetricos entre ellos, es decir se encontraban en las mismas filas, y que eran completamente aleatorios, es decir no dependian de alguna otra columna del Dataframe. Debido a la existencia de una cantidad importante de valores ausentes, alrededor de un 10% de las filas del Dataframe, se procedio a llenar estos con la media o mediana según corresponda de la columna más representativa, para el caso de "days_employed" se decidio utilizar "dob_years" para completar los valores debido a la relación entre la edad de la persona y la cantidad de experiencia que pueda tener, en este caso se utilizó la mediana puesto que existian valores extremos que alteraban los valores de la media. Con respecto a "total_income" se escogio el tipo de empleo como categoría para completar los datos debido a la relación directa existente entre las dos categorías y se utilizó la mediana debido a la posible existencia de valores extremos propios del tipo de categoría.

 Continuando con los valores duplicados, se concluyó que estos no eran coincidencias de datos ya que aunque no existe en los registros una columna que haga referencia a un identificador único para cada persona, la cantidad de parametros especificos distintos tales como cantidad de dias de experiencia o ingreso total son demasiado especificos como para que valores iguales en todas las categorias sean por coincidencia. Para halllar los duplicados se utilizó el metodo duplicated() y se eliminaron con el metodo drop_duplicated(), disminuyendo el conjunto de datos en un 0,3%, lo que significaba alrededor de 65 filas duplicadas. Las posibles razones para la existencia de estos datos pueden ser de caracter humano en el que se hayan repetido algunos clientes al realizar la obtención de información o al ingresarlos al sistema.

 Con respecto a los cambios del tipo de datos se encontrarón problemas comunes como diferencias de nomenclaturas en una columna las cuales fueron corregidas con los métodos str.lower() y replace(). Otros error más importantes fue la presencia de 0 en la edad de las personas lo cual no posee sentido lógico, este se puede atribuir a un error humano debido a la baja cantidad de filas con este error. Se decidio reemplazar los valores con la mediana de las edades, descartando la media debido a que su valor resultante no era un número entero. El último gran problema que existia en el Dataframe fueron los valores negativos en la columna de "days_employed", como el anterior esto no posee sentido logico pero a diferencia de ese caso alrededor de un 75% de las filas poseian este error por lo que al ser de una magnitud tan grande se asume que proviene de algún problema técnico en el ingreso de los datos. Se decidio reemplazar estos valores por unos de la misma amgnitud pero de valor positivo.

 Para finalizar, abordando el cuestionamiento principal del proyecto, es decir la influencia de ciertos parámetros en el cumplimiento del pago de un crédito podemos concluir lo siguiente:

1. ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo? Podemos concluir que existe una correlación entre las personas que poseen hijos y los que no en el cumplimiento del pago de créditos encontrándose a favor de estos últimos, lo que no se puede confirmar es que si existe una relación entre el número de hijos y la tasa de cumplimiento, ya que no se logró encontrar un patrón claro ya sea por la no existencia de este o por la falta de más datos para personas con una mayor cantidad de hijos.


2. ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo? Con respecto a este punto no se encontró una relación directa en este parámetro, aunque dos subcategorías del mismo presentan una tendencia más alta al incumplimiento del pago que el resto, personas no casadas y con unión civil, no existe una relación directa entre estas como para justificar la conexión. Se concluye que la razón por que estas categorías estén por sobre la tasa general es que se ven influenciadas indirectamente por otra característica tal como la edad de la persona.


3. ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo? En este apartado se puede ver una mayor tasa de incumplimiento en las personas de ingresos medios, pero esto puede deberse principalmente a que en estas categorías se aloja la mayor cantidad de datos del estudio por lo que al verse sobre representada en comparación con personas de mayores o menores ingresos es que se da este resultado.


4. ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo? De la misma manera que el estado civil, encontramos dos categorías que presentan una tasa de incumplimiento por sobre la general, la compra de un vehículo y la educación, pero ambas no están relacionadas directamente entre ellas por lo que se puede atribuir de la misma forma a que se ven influenciadas por otro parámetro como la edad, el cual seria el que está directamente relacionado con el incumplimiento del pago.